# This notebook is meant to be a demo of what can be done with intake on ref-marc datas

In [ ]:
import intake
import dask_hpcconfig
from distributed import Client
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf
import os

Detail of the imports:
-----------------------------------------------------------------------------------------------  
    Intake    
>- Handle the catalog contained in the yaml files.

    dask_hpcconfig    
>- Parrallelize your tasks on an hpc like datatrmor.

    Distributed import Client    
>- Client is creating the dask client

    Xarray    
>- Xarray makes operation on datasets and creates labbeled multidimensional arrays

    hvplot.xarray
>- Plotting 

    geoviews.features
>- Plotting that enables some features on maps 

    os
>- Can handle operating system dependant functionnality 

In [ ]:
cluster = dask_hpcconfig.cluster("datarmor-local")
cluster.scale(14)
client = Client(cluster)
client

To vizaulise the dask-lab extension you need to go the the dask-lab tab and fill it wih :
>- /user/{ JUPYTERHUB-USERNAME }/proxy/{ PORT-NUMBER }

In [ ]:
path = "/home/datawork-lops-iaocea/catalog/kerchunk/satellite/" 
files = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
files.sort()
len(files)

In [ ]:
def list_param(cat, param):
    return cat.metadata["parameters"][param]["allowed"]

def allowed_param(cat):
    return cat.metadata["parameters"]

def allowed_years(data):
    path = "/home/datawork-lops-iaocea/catalog/kerchunk/satellite/" + data
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    files.sort()
    return files

def year_concat(cat, source, region):
    concat = [cat[source](region=region, filename=i).to_dask() 
              for i in allowed_years(region)]

    return xr.concat(
        concat, 
        dim="time", 
        coords="minimal", 
        data_vars="minimal", 
        compat="override"
    )

Detail of the functions:
-----------------------------------------------------------------------------------------------  
    list_param(cat, param)    
>- Takes 2 arguments: cat = An opened catalog, param = The name of a parameter.
>- Returns a ptyhon list containing all the parameters.

    allowed_param(cat)    
>- Takes 1 argument: cat = An opened catalog
>- Returns a python dict containing the allowed values of each parameters and their description

    Allowed_years(data)    
>- Takes 1 argument: data = A region of data.
>- Returns a pythton list containing all the files stored in the directory. The combined jsons are stored in a directory named after the region (Same name than in the ref-marc directory)  
/!\ In this notebook, it will only work with ref-marc data because the json files are located at /home/datawork-lops-iaocea/catalog/kerchunk/ref-marc/. To use it to another directory, update the variable "path" in the function

    year_concat(cat, source, region)    
>- Takes 3 arguments: cat = An opened catalog, source = the source from this catalog, region = the region where the data comes from
>- Returns: A dataset concatenated over the time dimension


In [ ]:
cat = intake.open_catalog("/home/datawork-lops-iaocea/catalog/intake/satelite.yaml")

#### To open the catalog you can use the cell above, you just need to give the path to an intake catalog.

In [ ]:
regions = list_param(
    cat, "region"
)

### This tab is here to test the file and see how long it takes to be concatenated, if everything works as it should and measure the performance

Tested on datarmor with the spark notebook







|    REGION                                             |  OPEN DATASET  |    COMPUTE      |     PLOT    |
|    :------------------------------------              |:--------------:|:---------------:|:-----------:|
|   osisaf_public_l3_east_atlantic_west_indian_meteosat |Works           |Works            |Works        |
|   osisaf_public_l3_naravhrr_metop_a                   |Works           |Works            |Works        |
|   osisaf_public_l3_nar_avhrr_noa_19                   |Works           |Works            |Works        |
|   osisaf_public_l3_west_atlantic_east_pacific_goes    |Works           |Works            |Works        |
|                                                       |                |                 |             |



In [ ]:
region = regions[3]
regions

In [ ]:
ds = cat.satellite(region=regions[0]).to_dask()
ds

In [ ]:
da = ds.sea_surface_temperature 

In [ ]:
da.unify_chunks().hvplot.quadmesh(
     x="lon", y="lat", rasterize=True, geo=True, cmap="BuPu"
) * gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")